In [2]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

In [60]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [61]:
ratings['movieId'].value_counts()

356       329
318       317
296       307
593       279
2571      278
         ... 
5986        1
100304      1
34800       1
83976       1
8196        1
Name: movieId, Length: 9724, dtype: int64

In [95]:
np.mean(ratings[ratings['movieId'] == 193609].rating)

4.0

In [72]:
movieId_ratings = pd.DataFrame()
print(max(ratings['movieId']))
movieId = []
rating = []
for i in tqdm(range(1, 193610)):
    var = np.mean(ratings[ratings['movieId'] == i].rating)
    if var != 'Nan':
        movieId.append(i)
        rating.append(var)
movieId_ratings['movieId'] = movieId
movieId_ratings['ratings'] = rating

193609


In [76]:
movieId_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193609 entries, 0 to 193608
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   movieId  193609 non-null  int64  
 1   ratings  9724 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 3.0 MB


In [77]:
movieId_ratings.dropna(inplace=True)

In [78]:
movieId_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9724 entries, 0 to 193608
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9724 non-null   int64  
 1   ratings  9724 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 227.9 KB


In [79]:
movies_with_ratings = movies.join(movieId_ratings.set_index('movieId'), on='movieId')

In [138]:
movies_with_ratings.dropna(inplace=True)
movies_with_ratings

,movieId,title,genres,ratings
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143
4,5,Father of the Bride Part II (1995),Comedy,3.071429
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,4.000000
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,3.500000
9739,193585,Flint (2017),Drama,3.500000
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,3.500000


Подготовили датасет со сред. рейтингом фильмов.  
Теперь займемся подготовкой жанров. 

Делим данные на тестовые и учебные.

In [139]:
from sklearn.model_selection import train_test_split

In [140]:
def change_string(s):
    return " ".join(s.replace(' ', '').replace('-','').split('|'))

In [141]:
 movie_genres = [change_string(g) for g in movies_with_ratings.genres.values]

In [142]:
 count_vect = CountVectorizer()

In [143]:
X_train_counts = count_vect.fit_transform(movie_genres)

In [144]:
X_train_counts.toarray()

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [145]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [129]:
# X_train_tfidf = X_train_tfidf.toarray()

In [146]:
y = movies_with_ratings['ratings'] 

In [156]:
X_test, X_train, y_test, y_train = train_test_split(X_train_tfidf, y, test_size=0.9, random_state = 1000)

In [157]:
from sklearn.linear_model import LinearRegression
LinR = LinearRegression()

In [158]:
X_train.toarray()

array([[0.        , 0.        , 0.        , ..., 0.54781407, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.56455748, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [159]:
Lin_reg =  LinearRegression().fit(X_train, y_train) 

Предсказания

In [160]:
y_pred = Lin_reg.predict(X_test)

In [166]:
len(y_test)

972

In [167]:
print("Final rmse value is =",np.sqrt(np.mean((y_test-y_pred)**2)))

Final rmse value is = 0.8150125644907299
